## Retrieval Augmented Generation (RAG) chatbot on a custom PDF dataset

<img src="images/RAG-diagram-dell-technologies.png" alt="Alternative text" />

### Summary

- Retrieval augmented generation (RAG) is a method of augmenting the vectors of a model with the contents of a vector database created from custom content like documents and pdf files.  This is a very fast, low impact method that combines proven off the shelf models (Llama2 in this case) with your custom documents, providing a human-like response to questions based on the content in your document repo.
- RAG details: https://docs.langchain.com/docs/use-cases/qa-docs
- Llama2 details: https://huggingface.co/docs/transformers/main/model_doc/llama2

- This does not require time-consuming training runs since the contents of the documents are embedded in a vector database that is outside the model and none of the model weights are being modified.

- The vector database (Chromadb) in this case can be created and stored on prem and placed in block storage if necessary 

- This can all be updated real-time, allowing users to view the latest ground truth documents from your repo

- Example pdf documents were downloaded and stored on the host that the notebook server was running on.  Documents are from Dell Technologies Infohub https://infohub.delltechnologies.com/


### Prerequisites
- permission from Meta to use Llama-2 model  https://ai.meta.com/resources/models-and-libraries/llama-downloads/
- install poppler utils:  sudo apt-get install poppler-utils
- huggingface API token with write capability, insert into hugging face login token variable below
- pdf folder of files at the same file structure level as this notebook, "pdfs-dell-infohub", change the name as needed on your local host.
- pdf files inside folder as training content


### Notes
- Click Run -> Run all Cells to begin
- If it fails with CUDA out of memory, simply go to top navigation menu Kernel-> Restart Kernel and Clear output.  Then Run -> Run All Cells again.
- Tested successfully on Ubuntu 22.04 with Nvidia T4 GPU

### Install python libraries and applications

In [ ]:
## code to auto login to hugging face, avoid the login prompt

## your hugging face hub version has a lot of libraries that are used throughout.  Try to solve any dependencies with this if you experience errors. 
!pip install huggingface-hub==0.16.4

# get your account token from https://huggingface.co/settings/tokens
token = 'xxxxxxxxxxxxxxxxx'

from huggingface_hub import login
login(token=token, add_to_git_credential=True)

In [ ]:
!pip install cython

!pip install cchardet

In [ ]:
## ignore warnings about dependencies, as of publishing all of these versions work

In [ ]:
!pip install -U pip --quiet
!pip install torch --quiet
!pip install transformers --quiet
!pip install langchain --quiet
!pip install chromadb --quiet
!pip install pypdf --quiet
!pip install xformers --quiet
!pip install sentence_transformers --quiet
!pip install InstructorEmbedding --quiet
!pip install pdf2image --quiet
!pip install pycryptodome --quiet

In [ ]:
!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/ --quiet

### Import libraries

In [ ]:
import torch
from auto_gptq import AutoGPTQForCausalLM
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from pdf2image import convert_from_path
from transformers import AutoTokenizer, TextStreamer, pipeline

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

### Clear GPU memory from any previous runs
- assume Nvidia drivers installed
- When running notebooks over and over again, often much of the memory is still in the GPU memory allocated cache.  Depending on the size of the GPU, this might cause out of memory issues during the next run.  It is advised to clear out the cache, or restart the kernel.
- here we see multiple GPUs, the memory usage, any running processes and our CUDA version

In [ ]:
!nvidia-smi

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

### Clear the previous run vector database

This is optional, the vector db will be rebuilt.  For a completely fresh run you can delete the local folder.

In [ ]:
## remove chroma vector db local db folder from previous run

!rm -rf "db"

### Preview some of the PDF files

In [ ]:
pdf_images = convert_from_path("pdfs-dell-infohub/apex-navigator-for-multicloud-storage-solution-overview.pdf", dpi=100)
pdf_images[0]

In [ ]:
pdf_images = convert_from_path("pdfs-dell-infohub/h19751-aiml-platform-on-a-gpu-enabled-dell-apex-private-cloud-with-rancher-prime.pdf", dpi=100)
pdf_images[0]

### Load PDF documents as individual pages
- loader directory can be modified to suit your environment
- the command will look for a directory in the same path where the notebook is located
- this document directory and the files must be present before the run
- in this case we downloaded 40 pdf files.  The loader loads all of the pages associated with those files and offers what amounts to a page count.

In [ ]:
loader = PyPDFDirectoryLoader("pdfs-dell-infohub")
docs = loader.load()
len(docs)

### Use Huggingface Instructor model for content vector embeddings

- An instruction-finetuned text embedding model that can generate text embeddings tailored to any task
- https://huggingface.co/hkunlp/instructor-large
- this will be used to feed vectorized versions of our PDF content into the vector database

In [ ]:
embeddings = HuggingFaceInstructEmbeddings(
    model_name="hkunlp/instructor-large", model_kwargs={"device": DEVICE}
)

### Split our content into chunks
- Text splitting controls the granularity of chunks of text.  A smaller chunk size results in smaller chunks, but more of them.
- If you want more granular output from highly structured content, like code, you would need to make chunks smaller.
- If you are looking for output that contains whole sentences, like written text, then a larger chunk size works better. 
- Chunk overlap is the number of characters that should overlap between two adjacent chunks.
- https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/recursive_text_splitter

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
texts = text_splitter.split_documents(docs)
len(texts)

### Create content vector database on prem using Chroma db
- Parses pdf content into chunked encoded embeddings  https://www.trychroma.com/
- This is a sqlite db created in a local "db" folder in the same directory as this notebook
- This process takes a couple of minutes

In [ ]:
## takes a few minutes

vectordb = Chroma.from_documents(texts, embeddings, persist_directory="db")

### Llama-2  13 Billion parameter base pre-trained model
- pre-quantized model for memory optimimization
- requires at least 14Gb GPU memory
- https://huggingface.co/TheBloke/Llama-2-13B-chat-GPTQ
- code uses the 4bit revision:  4-bit, with Act Order and group size 128g. Uses even less VRAM than 64g, but with slightly lower accuracy
- gptq-4bit-128g-actorder_True   on a single T4 GPU, 4 bit is recommended.  If you want to run 8 bit you'll need more than 16Gb of GPU memory.

In [ ]:
## assign device to GPU
import os
os.environ['CUDA_VISIBLE_DEVICES']='0,1'

#import torch
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#model.to(device)

In [ ]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GPTQ"
model_basename = "model"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(
    model_name_or_path,
    revision="gptq-4bit-128g-actorder_True",
    model_basename=model_basename,
    use_safetensors=True,
    trust_remote_code=True,
    inject_fused_attention=False,
    device="cuda:0",
    quantize_config=None,
)

### Display text responses as being typed on screen

In [ ]:
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

In [ ]:
### ignore not supported warning

### Create pipeline to access the model API
- Huggingface pipeline library allows you to directly access the model with queries
- https://huggingface.co/docs/transformers/pipeline_tutorial
- do_sample: if set to True, this parameter enables decoding strategies such as multinomial sampling, beam-search multinomial sampling, Top-K sampling and Top-p sampling. All these strategies select the next token from the probability distribution over the entire vocabulary with various strategy-specific adjustments.

In [ ]:
text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
#    temperature=0,
    do_sample=True,
    top_p=0.95,
    repetition_penalty=1.15,
    streamer=streamer,
)

In [ ]:
llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0})

### Prompt template
- building a prompt template helps the model to better understand our question and frame the type of response we want.
- without a prompt template, the LLM can only be guided by the question, and that will not produce a well-formatted, full response.  We want to enable the LLM to give us the desired format by instructing it how to respond.
- the two prompts here tell it how to behave, and then tell it how to respond to the question.
- the function combines both prompts.

In [ ]:
DEFAULT_SYSTEM_PROMPT = """
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something incorrectly. If you don't know the answer to a question, please don't share false information.
""".strip()


def generate_prompt(prompt: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT) -> str:
    return f"""
[INST] <<SYS>>
{system_prompt}
<</SYS>>

{prompt} [/INST]
""".strip()

In [ ]:
SYSTEM_PROMPT = "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer."

template = generate_prompt(
    """
{context}

Question: {question}
""",
    system_prompt=SYSTEM_PROMPT,
)

In [ ]:
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

### Use Langchain library to link and retrieve similar document chunks from vectordb
- document content was chunked and vectorized and placed into the vector db
- here we will take the question, rank its similarity to the chunks of content, and retrieve those similar chunks
- the highest ranking, most similar chunks will be fed to the LLM along with the question

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectordb.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

### Prompts and responses
- Now let's ask it some questions.
- Questions will be fed to the chain, chunked data that is similar will be collected and then passed in the chain to the LLM for a response that is framed within our prompt template.

### Prompt #1

In [ ]:
result = qa_chain("Does apex block storage support multi availability zones?")

### source content location in original PDF

In [ ]:
print(result["source_documents"][0].page_content)

### Prompt #2

In [ ]:
result = qa_chain("provide a curl example of code to authenticate my powerflex")

### source content location in original PDF

In [ ]:
print(result["source_documents"][0].page_content)

### Prompt #3

In [ ]:
result = qa_chain("do you have any case studies where Intel GPU processors are used for AI?")

### source content location in original PDF

In [ ]:
print(result["source_documents"][0].page_content)

## References
- Code example for this notebook and others at:  https://github.com/dell-examples
- Dell Technologies Infohub:  https://infohub.delltechnologies.com/
- original version repo:  https://github.com/curiousily/Get-Things-Done-with-Prompt-Engineering-and-LangChain  
